In [1]:
import sys
import time

import numpy as np
from ordered_set import OrderedSet as oset

sys.path.append("..")
from filesplitter import subjects, ilp
from filesplitter.graph import group_by_scc, group_by_wcc, group_edges_by

In [2]:
DS = subjects.load_subject(subjects.ANDROID_BASE_TEXT_VIEW)
entities_df = DS.entities_df()
deps_df = DS.deps_df()
target_deps_df = DS.target_deps_df
edges = oset((r["src_id"], r["tgt_id"]) for _, r in deps_df.iterrows())

In [22]:
entities_df["foo"] = entities_df.groupby("block_name").ngroup()

In [3]:
# Create a "name_id" for each entity that groups targets according to their name
entities_df["name_id"] = entities_df.groupby("name").ngroup()

In [4]:
# Create a "strong_id" for each entity that groups targets according the strongly connected componant of their name
name_edges = group_edges_by(edges, entities_df["name_id"])
entities_df["strong_id"] = group_by_scc(entities_df["name_id"], name_edges)

In [5]:
# Create an "weak_id" for each entity that groups targets according the weakly connected componant of their strong_id
strong_edges = group_edges_by(edges, entities_df["strong_id"])
entities_df["weak_id"] = group_by_wcc(entities_df["strong_id"], strong_edges)

In [6]:
def get_entity_weight(id: int) -> int:
    kind = entities_df.loc[id]["kind"]
    return 0 if kind == "file" else 1

def get_strong_weight(strong_id: int) -> int:
    ids = entities_df[entities_df["strong_id"] == strong_id].index
    return sum(get_entity_weight(id) for id in ids)

def get_strong_loc(id: int) -> int:
    return entities_df[entities_df["strong_id"] == id]["loc"].sum()

In [7]:
# IDEA: The stopping criteria is based on density
# If `active` has too many `n_edges / n_nodes` then we stop
# Alternatively, if the `cut_weight / n_edges` or `cut_weight / n_nodes` is too high, then we stop

In [23]:
entities_df

,parent_id,name,kind,start_row,end_row,name_id,strong_id,weak_id,block_name,block_id,foo
id,,,,,,,,,,,
492564,1323,LOG_TAG,field,368.0,368.0,53,0,0,W0AAABBBB,31,31
492565,1323,DEBUG_EXTRACT,field,369.0,369.0,13,1,0,W0BAAAAAB,65,65
492566,1323,DEBUG_CURSOR,field,370.0,370.0,12,2,0,W0AABBAB,42,42
492567,1323,TEMP_POSITION,field,372.0,372.0,71,3,0,W0BBAAB,78,78
492568,1323,XMLTypefaceAttr,annotation,377.0,379.0,78,4,1,W1,81,81
...,...,...,...,...,...,...,...,...,...,...,...
815562,None,tests/VoiceInteraction/src/com/android/test/vo...,file,NaN,NaN,1290,1296,0,W0AAAABBBA,4,4
815601,None,tests/VoiceInteraction/src/com/android/test/vo...,file,NaN,NaN,1291,1297,0,W0AAAABBBA,4,4
815652,None,tests/WallpaperTest/src/com/example/wallpapert...,file,NaN,NaN,1292,1298,0,W0AAABBAABAAA,17,17


In [9]:
def cluster(edges: set[tuple[int, int]], active: set[int], name: str) -> dict[int, str] | None:
    active_edges = set((a, b) for a, b in edges if a in active and b in active)
    
    density = len(active_edges) / len(active)
    timestamp = time.strftime("%H:%M:%S", time.localtime())
    prefix = f"[{name}]".ljust(18) + f" ({timestamp})   "
    info = f"{len(active_edges)} edges and {len(active)} nodes = {density:0.4f} density"
    print(prefix + f"Starting... ({info})", end="\t")

    default_res = {i: name for i in active}

    if sum(get_strong_weight(strong_id) for strong_id in active) <= MAX_WEIGHT:
        print("Aborted. Weight under threshold.")
        return default_res

    def w(strong_id: int) -> int:
        if strong_id not in active:
            return 0
        return get_strong_weight(strong_id)

    # There are two ways to use `active`:
    # 1) Use ILP to bisect only the active elements
    #    - This might be faster.
    # 2) Use ILP to bisect all elements, but non-active elements are weighted to 0
    #    - This might produce better results.
    if USE_ALL:
        active_edges = edges

    start = time.perf_counter()
    cut_weight, labels = ilp.partition(list(active_edges), w, lambda i, j: 1, 2, EPS)
    if labels is None:
        print("Aborted. Failed to partition.")
        return default_res
    elapsed = time.perf_counter() - start
    print(f"Bisected with a cut weight of {cut_weight} in {elapsed:0.4f} secs.")

    active_A = active & {i for i, l in labels.items() if l == 0}
    active_B = active & {i for i, l in labels.items() if l == 1}
    res_A = cluster(edges, active_A, name + "A")
    res_B = cluster(edges, active_B, name + "B")
    return res_A | res_B


In [10]:
block_names = {}

for weak_id in range(entities_df["weak_id"].max() + 1):
    # The strong_ids inside the current weakly connected component (wcc)
    wcc_nodes = set(entities_df[entities_df["weak_id"] == weak_id]["strong_id"])
    wcc_edges = {(a, b) for a, b in strong_edges if a in wcc_nodes and b in wcc_nodes}
    block_names |= cluster(wcc_edges, wcc_nodes, name=f"W{weak_id}")

entities_df["block_name"] = [block_names.get(i) for i in entities_df["strong_id"]]
entities_df["block_id"] = entities_df.groupby("block_name").ngroup()

[W0]               (18:41:21)   Starting... (3123 edges and 1231 nodes = 2.5370 density)	Bisected with a cut weight of 214.0 in 3.9321 secs.
[W0A]              (18:41:25)   Starting... (2800 edges and 1055 nodes = 2.6540 density)	Bisected with a cut weight of 309.0 in 4.4522 secs.
[W0AA]             (18:41:29)   Starting... (2348 edges and 914 nodes = 2.5689 density)	Bisected with a cut weight of 326.0 in 4.4339 secs.
[W0AAA]            (18:41:34)   Starting... (1925 edges and 805 nodes = 2.3913 density)	Bisected with a cut weight of 323.0 in 2.8545 secs.
[W0AAAA]           (18:41:37)   Starting... (63 edges and 95 nodes = 0.6632 density)	Bisected with a cut weight of 23.0 in 0.5309 secs.
[W0AAAAA]          (18:41:38)   Starting... (4 edges and 18 nodes = 0.2222 density)	Bisected with a cut weight of 4.0 in 0.4453 secs.
[W0AAAAAA]         (18:41:38)   Starting... (1 edges and 4 nodes = 0.2500 density)	Aborted. Weight under threshold.
[W0AAAAAB]         (18:41:38)   Starting... (2 edges

## Validation

In [11]:
from collections import defaultdict
from random import shuffle

In [12]:
def count_blocks_touched(partition: dict[int, int], user_touches: set[int]) -> int:
    return len({partition[id] for id in user_touches})

def avg_blocks_touched_by_user(partition: dict[int, int], touches: dict[str, set[int]]) -> float:
    return np.average([count_blocks_touched(partition, t) for _, t in touches.items()])

def get_sizes(partition: dict[int, int]) -> list[int]:
    inverted = defaultdict(set)
    for entity, block in partition.items():
        inverted[block].add(entity)
    return list(sorted((len(x) for x in inverted.values()), reverse=True))

def rand_partition(sizes: list[int], entities: set[int]) -> dict[int, int]:
    rand_order = list(entities)
    shuffle(rand_order)
    partition = {}
    curr = 0
    for block, size in enumerate(sizes):
        for entity in rand_order[curr:curr+size]:
            partition[entity] = block
        curr += size
    return partition

In [13]:
targets_df = entities_df.loc[~(entities_df["kind"] == "file")].copy()
partition = {k: v for k,v in targets_df["block_id"].items()}

In [14]:
touches = defaultdict(set)
for _, row in DS.touches_df.iterrows():
    touches[row["author_email"]].add(row["entity_id"])

In [15]:
avg_blocks_touched_by_user(partition, touches)

3.5294117647058822

In [16]:
sizes = get_sizes(partition)
entities_set = set(partition.keys())

In [17]:
trials = [avg_blocks_touched_by_user(rand_partition(sizes, entities_set), touches) for _ in range(5_000)]
print(np.average(trials))

4.522923529411765
